In [1]:
import cv2
import numpy as np
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import itertools


In [2]:
def show(img, offset=1):
    shape = img.shape
#     offset = 1
    s = 0
    if shape[0] > 1000:
        s = int(10/(int(str(shape[0])[0])+offset))*0.1
    elif shape[1] > 1000:
        s = int(10/(int(str(shape[1])[0])+offset))*0.1
    if s:
        img = cv2.resize(img, None, fx=s, fy=s)
    cv2.imshow("Img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
base = cv2.imread('2.jpg')

In [4]:
df = pd.read_csv('product_export_2022-05-28-07-26-33.csv')

In [7]:
def read_image_from_url(url):
    img = Image.open(BytesIO(requests.get(url).content))
    im_arr = np.array(img)
    im_processed = cv2.cvtColor(im_arr, cv2.COLOR_RGB2BGR)
    return im_processed


def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

def convert_mockup(base, resized_book):
    mask = np.ones_like(base)
    k = apply_brightness_contrast(resized_book, 70, 0)
    insert_img = k.copy()

    x_offset=1007  
    y_offset=304

    x_end = x_offset+resized_book.shape[1]
    y_end = y_offset+resized_book.shape[0]
    mask[y_offset:y_end, x_offset:x_end] = insert_img 

    base[y_offset:y_end, x_offset:x_end] = insert_img 
    return base

In [8]:
urls = {}
for index, i in tqdm(enumerate(df['images'])):
    try:
        url = i.split('!')[0].strip()
        urls[index]= url
    except Exception as e:
        print(f"{i} -- e")


2924it [00:00, 147593.63it/s]

nan -- e
nan -- e
nan -- e
nan -- e
nan -- e
nan -- e
nan -- e
nan -- e


In [9]:
sliced_url_dict = dict(itertools.islice(urls.items(), 10))  # slice dictionary to n items

In [10]:
for index in sliced_url_dict:
    book = read_image_from_url(sliced_url_dict[index])    
    resized_book = cv2.resize(book, (984, 1372))  # resize to mockup book background
    mockup = convert_mockup(base, resized_book)
    df['images'][index] = 2

/home/sourav/anaconda3/envs/dsenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
df.head(15)['images']

0                                                     2
1                                                     2
2                                                     2
3                                                     2
4                                                     2
5                                                     2
6                                                     2
7                                                     2
8                                                     2
9                                                     2
10    https://staging.localmt.in/wp-content/uploads/...
11    https://staging.localmt.in/wp-content/uploads/...
12    https://staging.localmt.in/wp-content/uploads/...
13    https://staging.localmt.in/wp-content/uploads/...
14    https://staging.localmt.in/wp-content/uploads/...
Name: images, dtype: object

InvalidSchema: No connection adapters were found for "{0: 'https://staging.localmt.in/wp-content/uploads/2018/10/lifes-amazing-secrets-how-to-find-balance-and-purpose-in-your-life.jpg'}"

In [49]:
show(base)